In [2]:
import numpy as np
import queue
import copy
import matplotlib.pyplot as plt


In [3]:
# unit: hour
#即核酸检测有两个小时的检测窗口 eg. 9-11 a.m. 
#假设所有人都在这个区间进行检测👉后续可以优化
# simulation(hours)
#arrival rate (/hour) i.e. 2 per min
#service rate (/hour) 这里考虑改进前 即20s左右,每分钟三个人，每小时180个人
#我们假设在正式检测开始之前，已经有人开始排队👉即这里的‘begin', 是一个0-30之间的随机整数
begin = np.random.randint(0,30)
c = 1
total_time = 2
IAT_rate = 120
ST_rate = 180*c
rho = IAT_rate/ST_rate
print(begin)

17


In [4]:
# Initialize Parameters
# IAT  empty array for putting random number inter-arrival time
# St   ... for serving time rate
# AT: arrival times
# ST: serve time


qu = queue.Queue()
curr_process = None
IAT = []
ST = []
AT = []
wait_time = []
server_busy = False
list_wait = []
list_delay = []


In [21]:
#队伍长度

L_q = (IAT_rate*IAT_rate)/(ST_rate*(ST_rate-IAT_rate))
print(L_q)

1.3333333333333333


In [5]:
# total queue length in the system
L_s =  L_q + IAT_rate/ST_rate
print(L_s)

2.0


In [22]:
# utilization 
# 当只有一个核酸检测点时 lamda(effective) = lamda
# c bar (expected number of busy serves) = lamda/ miu
c_bar = IAT_rate/ST_rate
utilization = c_bar/c
print(utilization)

0.6666666666666666


In [5]:
# num_processes 即一共有多少个人在检测窗口里来做核酸检测

num_processes = int(np.random.poisson(IAT_rate)* total_time) 
total_people = num_processes + begin
num_processes_served = 0
print(num_processes)
print(begin)
print(total_people)

266
17
283


In [6]:
# Populate Inter-Arrival-Times (IAT)
# inter-arrival-time of 在核酸检测正式开始后来的人 保持不变
for i in range(num_processes):
    temp = np.random.exponential(1/IAT_rate)*60*60
    if i==0:
        IAT.append(0)
    else:
        IAT.append(int(temp - temp%1))


In [7]:
IAT_new = []

In [8]:
for i in range(total_people):
    temp = np.random.exponential(1/IAT_rate)*60*60
    if i<begin:
        IAT_new.append(0)
        i=i+1
    else: 
        IAT_new.append(int(temp-temp%1))

print(IAT_new)


[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 128, 7, 19, 13, 5, 21, 49, 18, 15, 16, 11, 15, 30, 15, 4, 25, 8, 1, 12, 4, 1, 43, 14, 4, 64, 50, 10, 27, 3, 2, 80, 6, 5, 15, 17, 39, 37, 21, 12, 26, 5, 28, 12, 12, 126, 91, 20, 18, 19, 82, 10, 32, 17, 20, 18, 6, 68, 76, 98, 19, 19, 21, 16, 13, 7, 28, 19, 16, 1, 56, 23, 43, 22, 20, 51, 9, 29, 25, 24, 15, 27, 26, 156, 25, 14, 81, 44, 5, 67, 1, 21, 5, 19, 3, 25, 14, 34, 31, 34, 0, 11, 0, 49, 2, 38, 6, 1, 7, 16, 37, 49, 58, 25, 41, 104, 49, 12, 21, 31, 2, 38, 5, 56, 28, 19, 33, 34, 19, 41, 6, 31, 12, 5, 112, 2, 8, 0, 17, 27, 16, 0, 0, 41, 6, 63, 14, 32, 13, 7, 41, 55, 33, 84, 23, 47, 24, 74, 27, 10, 3, 18, 39, 64, 36, 7, 22, 23, 5, 70, 90, 39, 32, 43, 1, 4, 60, 93, 3, 3, 5, 31, 60, 8, 48, 1, 44, 4, 131, 11, 107, 56, 44, 34, 60, 10, 30, 12, 13, 3, 64, 11, 45, 27, 2, 30, 17, 27, 61, 9, 7, 13, 5, 52, 69, 12, 103, 1, 4, 0, 159, 6, 13, 46, 18, 0, 62, 124, 19, 137, 69, 105, 2, 0, 5, 17, 84, 30, 17, 0, 47, 6, 57, 49, 26, 29, 16, 35, 10, 4, 41, 1

In [9]:
# Populate Service-Times (ST) (where ST[i]!=0)

while not len(ST) == total_people:
    temp = np.random.exponential(1/ST_rate)*60*60
    if not int(temp- temp%1)<1:
        ST.append(int(temp - temp%1))

print(ST)

[5, 6, 4, 18, 51, 21, 25, 13, 12, 10, 41, 15, 6, 8, 24, 8, 36, 3, 24, 36, 19, 13, 6, 11, 9, 1, 15, 60, 4, 9, 1, 22, 9, 9, 18, 3, 8, 13, 11, 9, 64, 28, 5, 87, 1, 21, 5, 25, 10, 2, 6, 12, 20, 41, 26, 1, 53, 11, 3, 37, 25, 14, 2, 2, 25, 4, 29, 2, 31, 2, 7, 8, 5, 17, 5, 1, 39, 55, 3, 15, 18, 30, 28, 8, 4, 15, 5, 2, 7, 4, 12, 23, 16, 2, 3, 1, 21, 2, 34, 63, 35, 19, 25, 10, 44, 46, 24, 4, 10, 22, 4, 3, 40, 5, 18, 42, 9, 21, 5, 2, 32, 19, 13, 8, 1, 4, 12, 14, 3, 2, 42, 6, 35, 25, 14, 7, 41, 39, 51, 4, 15, 21, 17, 21, 27, 25, 4, 5, 18, 3, 14, 5, 4, 8, 14, 30, 32, 29, 4, 4, 3, 3, 57, 23, 2, 7, 7, 1, 32, 31, 3, 85, 36, 7, 9, 2, 25, 12, 8, 5, 1, 7, 31, 24, 6, 4, 48, 12, 55, 4, 17, 9, 28, 29, 12, 28, 20, 7, 5, 10, 38, 41, 17, 4, 12, 19, 44, 24, 24, 33, 3, 17, 69, 9, 64, 64, 3, 8, 85, 1, 14, 12, 24, 9, 28, 3, 28, 18, 4, 28, 7, 22, 12, 21, 14, 10, 44, 14, 1, 11, 14, 25, 2, 10, 76, 38, 1, 32, 8, 5, 3, 11, 16, 2, 4, 8, 11, 31, 4, 46, 4, 4, 3, 19, 19, 7, 7, 13, 16, 12, 13, 62, 42, 1, 40, 2, 10, 6, 2, 3

In [29]:
# save a copy of ST

ST_copy = copy.deepcopy(ST)

In [49]:
AT_new = []

In [50]:
# Get Arrival-Times (AT) from IAT starting at t=0 
# 生成到达时间队列
# and initialize Waiting-Times to 0

for i in range(total_people):
    if i<begin:
        AT_new.append(0)
        i=i+1
    else:
        AT_new.append(IAT_new[i]+AT_new[i-1])
    wait_time.append(0)

print(AT_new)


[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 128, 135, 154, 167, 172, 193, 242, 260, 275, 291, 302, 317, 347, 362, 366, 391, 399, 400, 412, 416, 417, 460, 474, 478, 542, 592, 602, 629, 632, 634, 714, 720, 725, 740, 757, 796, 833, 854, 866, 892, 897, 925, 937, 949, 1075, 1166, 1186, 1204, 1223, 1305, 1315, 1347, 1364, 1384, 1402, 1408, 1476, 1552, 1650, 1669, 1688, 1709, 1725, 1738, 1745, 1773, 1792, 1808, 1809, 1865, 1888, 1931, 1953, 1973, 2024, 2033, 2062, 2087, 2111, 2126, 2153, 2179, 2335, 2360, 2374, 2455, 2499, 2504, 2571, 2572, 2593, 2598, 2617, 2620, 2645, 2659, 2693, 2724, 2758, 2758, 2769, 2769, 2818, 2820, 2858, 2864, 2865, 2872, 2888, 2925, 2974, 3032, 3057, 3098, 3202, 3251, 3263, 3284, 3315, 3317, 3355, 3360, 3416, 3444, 3463, 3496, 3530, 3549, 3590, 3596, 3627, 3639, 3644, 3756, 3758, 3766, 3766, 3783, 3810, 3826, 3826, 3826, 3867, 3873, 3936, 3950, 3982, 3995, 4002, 4043, 4098, 4131, 4215, 4238, 4285, 4309, 4383, 4410, 4420, 4423, 4441, 4480, 4544, 4580, 4587, 46

In [51]:
# simulate M/M/1 queue

for i in range(total_time*60*60):    
    if server_busy:
        for item in list(qu.queue):
            wait_time[item] = wait_time[item] + 1
        ST[curr_process] = ST[curr_process] - 1
        if ST[curr_process] == 0:
            server_busy = False
            num_processes_served = num_processes_served + 1

    for j in range(num_processes):
        if i== AT[j]:
            qu.put(j)

    if not server_busy and not qu.empty():
        curr_process = qu.get()
        server_busy = True
        


In [52]:
# 总共等待时间
# 总共delay时间 = total time in the system 
# delay = waiting time + serve time
sum_wait = 0
sum_delay = 0

In [53]:
for i in range(num_processes_served):
    sum_wait = sum_wait + wait_time[i]
    sum_delay = sum_delay + wait_time[i] + ST_copy[i]
    
if num_processes_served == 0:
    list_wait.append(0)
    list_delay.append(0)
else:
    list_wait.append(sum_wait/(num_processes_served*60*60))  
    list_delay.append(sum_delay/(num_processes_served*60*60))


In [55]:
#平均等待时间 in hour
print(sum_wait/(num_processes_served*60*60))
print(sum_delay/(num_processes_served*60*60))

# 平均等待时间 in second
print(sum_wait/num_processes_served)

0.007080610021786492
0.01207843137254902
25.49019607843137


In [20]:
# 单位仍选择为小时
# total cost = service cost + waiting cost 
# ETC(x) = EOC(x) + EWC (x)
# average salary of Beijing: 9227 per month
# 假设每月抛去四个周末，有效工作日为31-8=23天
# 假设每天有效工作时长为8小时
# 根据以上两个假设算时薪
worktime = 23*8
nurse_salary = 6000
salary = 9227
EOC = (nurse_salary/worktime) * total_time*c 
EWC = (salary/worktime)* L_s
ETC = EOC + EWC

print(ETC)


165.51086956521738


In [23]:
print((6000/worktime)*2)
print(salary/worktime)

65.21739130434783
50.14673913043478
